## GPT가 인터넷 검색을 사용해서 질문에 답하는 ReAct 기반 AI 에이전트

### 1. API 키 등록

In [ ]:
!pip install python-dotenv

In [1]:
# API 키 설정
import os
from dotenv import load_dotenv

load_dotenv()

open_api_key = os.getenv('OPEN_API_KEY')
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

SerpAPI는 Google 검색을 API로 사용할 수 있게 해주는 서비스이다.

### 2. LangChain Hub에서 ReAct 프롬프트 가져오기

In [2]:
# LangChain Hub 가져오기
from langchain import hub

# LangChain Hub에서 ReAct의 프롬프트 가져오기
prompt = hub.pull("hwchase17/react", api_key='LangSmith API Key')

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


LangChain의 Hub에서 ReAct 프롬프트 템플릿을 불러온다.
- `"hwchase17/react"`: LangChain의 ReAct 전략 프롬프트
- `LangSmith API Key`: LangSmith 계정 키 (필요시 등록해야 함)

| ReAct 전략은 Reasoning + Acting의 약자. LLM이 스스로 생각 -> 도구 사용 -> 최종 응답을 반복하며 문제를 해결하는 프레임워크이다.

### 3. LLM(OpenAI) + Search 도구 설정

In [4]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.5 MB/s eta 0:00:00


In [5]:
# OpenAI 가져오기
from langchain_openai import OpenAI

# 사용할 대형 언어 모델 선택
llm = OpenAI()

OpenAI GPT 모델을 사용한다.

In [7]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [11]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=91db7f49385d32abe51211586117a98d3f5592cfefc13c0d65e24873eb814b65
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [12]:
# SerpAPIWrapper를 통해 도구 모듈 가져오기
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool

# SerpAPIWrapper 인스턴스화
search = SerpAPIWrapper()

# 도구 목록 준비
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="대형 언어 모델이 관련 지식이 없을 때 지식 검색에 사용됩니다."
    ),
]

검색 도구(SerpAPI)를 Tool 객체로 구성
- `func`: 실행할 함수 (`search.run`)
- `description`: 이 도구를 사용할 상황 설명 (LLM이 사용 결정에 참고함)

### 4. ReAct 에이전트 생성

In [14]:
# create_react_agent 기능 가져오기
from langchain.agents import create_react_agent

# ReAct 에이전트 생성
agent = create_react_agent(llm, tools, prompt)

앞에서 설정한 LLM + 도구 + 프롬프트를 기반으로 ReAct 에이전트 생성
- 이 에이전트는 프롬프트 지침에 따라 질문에 대해 도구를 사용하거나 직접 답변함

### 5. AgentExecutor로 실행

In [15]:
# AgentExecutor 가져오기
from langchain.agents import AgentExecutor

# 에이전트와 도구를 전달하여 AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

에이전트를 실행할 수 있는 인터페이스를 생성
- `verbose=True`: 도구 사용 로그를 출력해서 추론 과정을 확인할 수 있음

In [16]:
# AgentExecutor를 호출하여 입력 데이터 전달
print("첫 번째 실행 결과:")
agent_executor.invoke({"input": "현재 인공 지능 에이전트의 최신 연구 진전은 무엇입니까?"})

첫 번째 실행 결과:


> Entering new AgentExecutor chain...
 인공 지능의 최신 연구 진전은 무엇인지에 대해 알아보기 위해 지식 검색을 해야합니다.
Action: Search
Action Input: "latest research progress in artificial intelligence agents"["2025's agents will be fully autonomous AI programs that can scope out a project and complete it with all the necessary tools they need and with no help from ...", 'Increased Autonomy: AI agents will become more capable of operating independently and making complex decisions with minimal human intervention.', 'Nearly 90% of notable AI models in 2024 came from industry, up from 60% in 2023, while academia remains the top source of highly cited research. Model scale ...', 'A multi-agent AI system built with Gemini 2.0 as a virtual scientific collaborator to help scientists generate novel hypotheses and research proposals.', '22, 2025 — Researchers have developed a new artificial intelligence (AI) technique that brings machine vision closer to how the human brain processes images.', 'This article high

{'input': '현재 인공 지능 에이전트의 최신 연구 진전은 무엇입니까?',
 'output': '최근 인공 지능 에이전트의 개발과 연구에는 다양한 발전이 있었으며, 이는 다양한 산업에 적용될 수 있는 기술을 포함하고 있습니다. 마이크로소프트사의 빌드 컨퍼런스를 비롯해 다양한 기업들이 AI 에이전트를 중심으로 발표하고 있으며, 미래에는 인공 지능 에이전트가 많은 분야에서 인간의 역할을 대체할 수 있을 것으로 예상됩니다.'}

질문을 입력하면 에이전트가 실행되어 **필요시 검색 도구를 사용하고, 최종 답변을 생성**한다.

### 두번 호출하는 이유?

In [17]:
print("두 번째 실행 결과:")
agent_executor.invoke({"input": "현재 인공 지능 에이전트의 최신 연구 진전은 무엇입니까?"})

두 번째 실행 결과:


> Entering new AgentExecutor chain...
 최신 연구를 살펴보고 무엇을 찾을지 고민해야합니다.
Action: Search
Action Input: '인공 지능 에이전트 최신 연구'['KDB미래전략연구소는 AI 에이전트 개발 동향 및 국내 경쟁력을 분석한 보고서를 발표하였다. - 가트너를 비롯한 주요 시장조사기관들은 2025년 가장 ...', 'AI 에이전트는 단순한 질문 대답 시스템이 아닌, 사용자의 의도를 이해하고 목표를 달성하기 위한 종합적인 솔루션입니다. 첫 번째로, 의도 인식(intent ...', 'AI 에이전트(AI Agent)는 생성형 AI 진화의 최신 단계로, 인간의 행동을 모방하고 자율적으로 작업을 수행할 수 있는 기술을 의미합니다. AI로봇 ...', 'AI 에이전트의 유용한 10가지 사례 연구 · 1. 웨이버 그룹의 리드 생성 봇 · 2. 루비랩스의 고객 서비스 봇 · 3. Botpress 경쟁 인텔리전스 봇 · 4. Pinterest ...', 'LLM을 탑재한 AI 에이전트는 여러 가지 툴과 로직들을 활용해 훈련 데이터(training data)에 없는 정보를 외부 시스템이나 툴을 통해 얻을 수 있으며, ...', '사람을 대신해 미션을 해결해주거나 주요 과업을 자동화하여 돕는 AI. 사용자와의 의사소통을 통해 의도를 정확히 파악하고 원하는 결과를 만들어 ...', '- AI 에이전트는 정해진 목표를 달성하기 위해 상황을 파악하고, 워크플로우를 계획하며, 외부 데이터와 분석 도구를 활용해 자율적으로 작업을 수행하는 ...', '단순한 명령 수행이나 챗봇 기능을 넘어, 사용자 목표를 인식하고 계획을 수립한 뒤 도구를 활용해 과업을 수행하는 능동형 주체로 진화하고 있다는 것이 ...', "'AI 에이전트'는 사용자의 의도를 파악하고 주어진 문제를 자율적으로 해결할 수 있는 인공지능 시스템입니다. 영화 아이언맨에 등장하는 인공지능 비서 ' ...", "구글은 'AI 공동 과학자'의 성능을 검

{'input': '현재 인공 지능 에이전트의 최신 연구 진전은 무엇입니까?',
 'output': '최신 연구 진전은 자율 에이전트와 AI Agent와 RAG의 결합 등이 있으며, AI 조수가 당신을 대신해 연구를 진행하는 등 AI 서비스도 다각화되고 있음을 알 수 있습니다.'}

같은 질문을 두 번 실행하는 건:

- 에이전트가 매번 다른 reasoning path(추론 경로) 를 택하는지 확인하려는 테스트일 수 있다.
- 검색 결과나 응답에 변동성이 있는지 보는 용도

### 요약
이 코드는 다음을 구현한다:

> "GPT가 인터넷 검색을 사용해서 질문에 답하는 ReAct 기반 AI 에이전트"

- 생각하고
- 필요하면 도구(Search)를 사용하고
- 최종 답변을 생성함